In [17]:
from nbgeohash import nb_point_decode, nb_vector_decode
from pygeohash import point_decode
np_vector_decode = np.vectorize(point_decode)

# Create random geohashes

In [11]:
__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'

# performance imporvements depend on geohash precision so we're gonna fix it to 12 which is a very common precision
geohash = "".join(np.random.choice(list(__base32), size=12))

# performance improvements also depend on the number of geohashes (thanks to the super optimized numba loop)
geohashes = np.array([
    "".join(arr_hash)
    for arr_hash in np.random.choice(list(__base32), size=(100000, 12))
])

# Point-wise performance
That's a x4.5 gain in performance.
I couldn't get any better performances because the python version uses a dictionary which allows it to decode characters very fast (compared to anything possible in numba).

In [13]:
%%timeit
point_decode(geohash)

20.4 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
%%timeit
nb_point_decode(geohash)

4.48 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Vector-wise performance
That's a x11 gain in performance and we are comparing to an optimized vectorization from numpy here

In [18]:
%%timeit
np_vector_decode(geohashes)

2.09 s ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
nb_vector_decode(geohashes)

164 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
